# 9. Convolutional Neural Network with PyTorch
## 1. About Convolutional Neural Network

### 1.1 Transition From Feedforward Neural Network

#### 1 Hidden Layer Feedforward Neural Network
<img src="./images/nn1_new.png" alt="deeplearningwizard" style="width: 900px;"/>

#### Basic Convolutional Neural Network
- **feedforward neural network 이전에** **convolution**과 **pooling** layer를 추가시킴
- Layer with a **linear function & non-linearity**: **Fully connected layer**
<img src="./images/cnn1.png" alt="deeplearningwizard" style="width: 900px;"/>

### 1.2 One Convolutional Layer: High Level View
<img src="./images/cnn2.png" alt="deeplearningwizard" style="width: 900px;"/>
<img src="./images/cnn3.png" alt="deeplearningwizard" style="width: 900px;"/>
<img src="./images/cnn2.png" alt="deeplearningwizard" style="width: 900px;"/>
<img src="./images/cnn6-1.png" alt="deeplearningwizard" style="width: 900px;"/>
<img src="./images/cnn6-2.png" alt="deeplearningwizard" style="width: 900px;"/>

### 1.2 One Convolutional Layer: High Level View Summary
<img src="./images/cnn2.png" alt="deeplearningwizard" style="width: 900px;"/>

- kernel이 이미지를 슬라이딩하면서 **patch(receptive field) 하나에 2가지 연산**을 수행한다.
    1. Element-wise multiplication
    2. Summation


- 더 많은 **kernels** $=$ 더 많은 **feature map chnnels**
    - 입력에 대해서 **더 많은 정보**를 얻을 수 있다.
    

### 1.3 Multiple Convolutional Layers: High Level View
<img src="./images/cnn7-3.png" alt="deeplearningwizard" style="width: 900px;"/>

### 1.4 Pooling Layer: High Level View
- 2 Common Types
    - Max Pooling
    - Average Pooling

<img src="./images/cnn8n2.png" alt="deeplearningwizard" style="width: 900px;"/>

### 1.5 Multiple Pooling Layers: High Level View
<img src="./images/cnn7-3.png" alt="deeplearningwizard" style="width: 900px;"/>

### 1.6 Padding
<img src="./images/cnn9-4.png" alt="deeplearningwizard" style="width: 900px;"/>

### 1.7 Padding Summary
- **Valid** Padding (Zero Padding)
    - Output size < Input Size
- **Same** Padding
    - Output size = Input Size

### 1.8 Dimension Calculations
- $ O = \frac {W - K + 2P}{S} + 1$
    - $O$: output height,width
    - $W$: input height,width
    - $K$: filter size (kernel size)
    - $P$: padding
        - $ P = \frac{K - 1}{2} $
    - $S$: stride


#### Example 1: Output Dimension Calculation for Valid Padding
<img src="./images/cnn9-2.png" alt="deeplearningwizard" style="width: 900px;"/>
- $W = 4$
- $K = 3$
- $P = 0$
- $S = 1$
- $O = \frac {4 - 3 + 2*0}{1} + 1 = \frac {1}{1} + 1 = 1 + 1 = 2 $


#### Example 2: Output Dimension Calculation for Same Padding
<img src="./images/cnn9-3.png" alt="deeplearningwizard" style="width: 900px;"/>
- $W = 5$
- $K = 3$
- $P = \frac{3 - 1}{2} = \frac{2}{2} = 1 $
- $S = 1 $
- $O = \frac {5 - 3 + 2*1}{1} + 1 = \frac {4}{1} + 1 = 5$


## 2. Building a Convolutional Neural Network with PyTorch

### Model A: 
- 2 Convolutional Layers
    - Same Padding (same output size)
- 2 Max Pooling Layers
- 1 Fully Connected Layer
<img src="./images/cnn10-1.png" alt="deeplearningwizard" style="width: 900px;"/>

### Steps
- Step 1: Load Dataset
- Step 2: Make Dataset Iterable
- Step 3: Create Model Class
- Step 4: Instantiate Model Class
- Step 5: Instantiate Loss Class
- Step 6: Instantiate Optimizer Class
- Step 7: Train Model

### Step 1: Loading CIFAR-10 Train Dataset

**CIFAR-10 dataset Input size: (3,32,32)** <br>
**CIFAR-10 dataset classes: 10 (plane, car, bird, cat, deer, dog, frog, horse, ship, truck)** <br>
**Images from 1 to 9**

In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 100, shuffle = True, num_workers = 2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 100, shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### Step 2: Make Dataset Iterable

In [3]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)

### Step 3: Create Model Class
<img src="./images/cnn10-1.png" alt="deeplearningwizard" style="width: 900px;"/>

#### Output Formula for Convolution
- $ O = \frac {W - K + 2P}{S} + 1$
    - $O$: output height/length
    - $W$: input height/length
    - $K$: **filter size (kernel size) = 5**
    - $P$: **same padding (non-zero)**
        - $P = \frac{K - 1}{2}  = \frac{5 - 1}{2} = 2$
    - $S$: **stride = 1**
    
#### Output Formula for Pooling
- $ O = \frac {W}{K}$
    - W: input height/width
    - K: **filter size = 2**
    
<img src="./images/cnn10-2n.png" alt="deeplearningwizard" style="width: 900px;"/>

In [4]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(8*8*32, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.conv1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.conv2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

### Step 4: Instantiate Model Class

In [5]:
model = CNNModel()

### Step 5: Instantiate Loss Class
- Convolutional Neural Network: **Cross Entropy Loss**
    - _Feedforward Neural Network_: **Cross Entropy Loss**
    - _Logistic Regression_: **Cross Entropy Loss**
    - _Linear Regression_: **MSE**
    
   

In [6]:
criterion = nn.CrossEntropyLoss()

### Step 6: Instantiate Optimizer Class
- Simplified equation
    - $\theta = \theta - \eta \cdot \nabla_\theta $
        - $\theta$: parameters (our tensors with grad accumulation)
        - $\eta$: learning rate (how fast we want to learn)
        - $\nabla_\theta$: parameters' gradients
- Even simplier equation
    - `parameters = parameters - learning_rate * parameters_gradients`
    - **At every iteration, we update our model's parameters**

In [7]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

### Parameters In-Depth

In [8]:
print(model.parameters())

print(len(list(model.parameters())))

# Convolution 1: 16 Kernels
print(list(model.parameters())[0].size())

# Convolution 1 Bias: 16 Kernels
print(list(model.parameters())[1].size())

# Convolution 2: 32 Kernels with depth = 16
print(list(model.parameters())[2].size())

# Convolution 2 Bias: 32 Kernels with depth = 16
print(list(model.parameters())[3].size())

# Fully Connected Layer 1
print(list(model.parameters())[4].size())

# Fully Connected Layer Bias
print(list(model.parameters())[5].size())

<generator object Module.parameters at 0x0000021A7C22CAC0>
6
torch.Size([16, 3, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 2048])
torch.Size([10])


### Step 7: Train Model
- Process 
    1. **Convert inputs/labels to tensors with grad accumulation**
        - CNN Input: (3, 32, 32) 
        - Feedforward NN Input: (3, 32*32)
    2. Clear gradient buffets
    3. Get output given inputs 
    4. Get loss
    5. Get gradients w.r.t. parameters
    6. Update parameters using gradients
        - `parameters = parameters - learning_rate * parameters_gradients`
    7. REPEAT

In [ ]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        # Load images
        images = Variable(images)
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        
    
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in testloader:
                # Load images
                images = Variable(images)
                labels = Variable(labels)
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

### Model B: 
- 2 Convolutional Layers
    - Same Padding (same output size)
- 2 **Average Pooling** Layers
- 1 Fully Connected Layer
<img src="./images/cnn10-3.png" alt="deeplearningwizard" style="width: 900px;"/>
<img src="./images/cnn10-4.png" alt="deeplearningwizard" style="width: 900px;"/>

### Steps
- Step 1: Load Dataset
- Step 2: Make Dataset Iterable
- Step 3: Create Model Class
- Step 4: Instantiate Model Class
- Step 5: Instantiate Loss Class
- Step 6: Instantiate Optimizer Class
- Step 7: Train Model

In [10]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torch.utils.data import DataLoader
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train = True, download = True, transform = transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform = transform)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 2)

testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Average pool 1
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
     
        # Convolution 2
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Average pool 2
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(2048, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.conv1(x)
        out = self.relu1(out)
        
        # Average pool 1
        out = self.avgpool1(out)
        
        # Convolution 2 
        out = self.conv2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.avgpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

'''
STEP 4: INSTANTIATE MODEL CLASS
'''

model = CNNModel()

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        # Load images as tensors requiring grad accumulation
        images = Variable(images)
        labels = Variable(labels)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in testloader:
                # Load images as tensors requiring grad accumulation
                images = Variable(images)
                labels = Variable(labels)
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Files already downloaded and verified
Files already downloaded and verified
Iteration: 500. Loss: 1.8910136222839355. Accuracy: 34
Iteration: 1000. Loss: 1.617234230041504. Accuracy: 40
Iteration: 1500. Loss: 1.5818179845809937. Accuracy: 44
Iteration: 2000. Loss: 1.3989008665084839. Accuracy: 48
Iteration: 2500. Loss: 1.5171809196472168. Accuracy: 48
Iteration: 3000. Loss: 1.5169521570205688. Accuracy: 51


### Average Pooling Test Accuracy < Max Pooling Test Accuracy

### Model C: 
- 2 Convolutional Layers
    - **Valid Padding** (smaller output size)
- 2 **Max Pooling** Layers
- 1 Fully Connected Layer
<img src="./images/cnn10-5.png" alt="deeplearningwizard" style="width: 900px;"/>
<img src="./images/cnn10-6n.png" alt="deeplearningwizard" style="width: 900px;"/>

### Steps
- Step 1: Load Dataset
- Step 2: Make Dataset Iterable
- Step 3: Create Model Class
- Step 4: Instantiate Model Class
- Step 5: Instantiate Loss Class
- Step 6: Instantiate Optimizer Class
- Step 7: Train Model

In [0]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torch.utils.data import DataLoader
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train = True, download = True, transform = transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform = transform)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 2)

testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(800, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.conv1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.conv2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

'''
STEP 4: INSTANTIATE MODEL CLASS
'''

model = CNNModel()

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        # Load images as tensors require grad accumulation
        images = Variable(images)
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in testloader:
                # Load images as tensors requiring grad accumulation
                images = Variable(images)
                labels = Variable(labels)
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Files already downloaded and verified
Files already downloaded and verified
Iteration: 500. Loss: 1.9381294250488281. Accuracy: 33
Iteration: 1000. Loss: 1.7005096673965454. Accuracy: 41
Iteration: 1500. Loss: 1.5323582887649536. Accuracy: 45
Iteration: 2000. Loss: 1.4167605638504028. Accuracy: 48
Iteration: 2500. Loss: 1.2144975662231445. Accuracy: 50
Iteration: 3000. Loss: 1.2880160808563232. Accuracy: 52


### Summary of Results

| Model A | Model B   | Model C | 
|------|------|
|   Max Pooling  | Average Pooling | Max Pooling |
| Same Padding | Same Padding | Valid Padding | <br>
| 54% | 51% | 52% | 


| All Models |
|------|
| INPUT $\rightarrow$ CONV $\rightarrow$ POOL $\rightarrow$ CONV $\rightarrow$ POOL $\rightarrow$ FC | 
| Convolution Kernel Size = 5 x 5 |
| Convolution Kernel Stride = 1 |
| Pooling Kernel Size = 2 x 2 |



## 3. Building a Convolutional Neural Network with PyTorch (GPU)
### Model A
<img src="./images/cnn10-1.png" alt="deeplearningwizard" style="width: 900px;"/>
<img src="./images/cnn10-2n.png" alt="deeplearningwizard" style="width: 900px;"/>

GPU: 2 things must be on GPU
- `model`
- `tensors with grad accumulation`

### Steps
- Step 1: Load Dataset
- Step 2: Make Dataset Iterable
- Step 3: Create Model Class
- **Step 4: Instantiate Model Class**
- Step 5: Instantiate Loss Class
- Step 6: Instantiate Optimizer Class
- **Step 7: Train Model**

In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torch.utils.data import DataLoader
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train = True, download = True, transform = transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform = transform)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 2)

testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(800, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.conv1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.conv2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

'''
STEP 4: INSTANTIATE MODEL CLASS
'''

model = CNNModel().cuda()

#######################
#  USE GPU FOR MODEL  #
#######################

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model.to(device)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        # images = Variable(images).to(device)
        # labels = Variable(labels).to(device)
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in testloader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # images = Variable(images).to(device)
                # labels = Variable(labels).to(device)
                images = Variable(images).cuda()
                labels = Variable(labels).cuda()
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Files already downloaded and verified
Files already downloaded and verified
Iteration: 500. Loss: 1.9880794286727905. Accuracy: 33
Iteration: 1000. Loss: 1.5586062669754028. Accuracy: 40
Iteration: 1500. Loss: 1.475795030593872. Accuracy: 45
Iteration: 2000. Loss: 1.394392490386963. Accuracy: 48
Iteration: 2500. Loss: 1.389755129814148. Accuracy: 49
Iteration: 3000. Loss: 1.3480387926101685. Accuracy: 52


## Model A에 몇가지 변화를 주었다.
- Optimizer: SGD -> Adam
- Learning rate: 0.01 -> 0.002
- CNNModel()에 Conv Layer, ReLU, MaxPooling 1개씩 추가

In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torch.utils.data import DataLoader
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train = True, download = True, transform = transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform = transform)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = 6
#num_epochs = n_iters / (len(trainset) / batch_size)
#num_epochs = int(num_epochs)

print(len(trainset), num_epochs)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 2)

testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Convolution 3
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        
        # Max pool 3
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(1024, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.conv1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.conv2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)

        # Convolution 3
        out = self.conv3(out)
        out = self.relu3(out)
        
        # Max pool 3
        out = self.maxpool3(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

'''
STEP 4: INSTANTIATE MODEL CLASS
'''

model = CNNModel().cuda()

#######################
#  USE GPU FOR MODEL  #
#######################

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model.to(device)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.002

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        #images = Variable(images).to(device)
        #labels = Variable(labels).to(device)
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in testloader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                #images = Variable(images).to(device)
                #labels = Variable(labels).to(device)
                images = Variable(images).cuda()
                labels = Variable(labels).cuda()
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Files already downloaded and verified
Files already downloaded and verified
50000 6
Iteration: 500. Loss: 1.0809932947158813. Accuracy: 58
Iteration: 1000. Loss: 1.0294924974441528. Accuracy: 64
Iteration: 1500. Loss: 0.9562081098556519. Accuracy: 68
Iteration: 2000. Loss: 0.9743576645851135. Accuracy: 69
Iteration: 2500. Loss: 0.6855459809303284. Accuracy: 70
Iteration: 3000. Loss: 0.6557455658912659. Accuracy: 70


# Tip

## - 어떻게 **인식률(Accuracy)**을 향상시킬까?
**1. Learning rate, Batch size, Epoch 등과 같은 Hyperparameter 값들을 조정한다.** <br>
**2. Layer를 깊게 쌓아본다. -> 너무 깊게 쌓으면 연산량이 많아져 높은 컴퓨팅 파워를 요구 -> 저사양의 환경에서는 매우 오래걸림** <br>
**3. Batch Normalization -> nn.BatchNorm2d()** <br>
**4. VGGNet, ResNet, GoogLeNEt 등과 같은 검증된 CNN Network를 사용한다. 단, torchvision에 있는 모델을 불러오지 말고 본인이 네트워크를 보고 네트워크를 수정해야 한다.**
5. 가중치 초기화(Weight Initialization): He Initialization, Xavier Initialization, ...
6. 데이터 전처리(Data Preprocessing)
7. 정규화(Regularization)
  
## - Conv block
<img src="./images/conv-block.png" alt="deeplearningwizard" style="width: 200px;"/>
<br>

 - model = nn.Sequential(<br>
    nn.Conv2d(input_channel, output_channel, kernel_size=3, stride=1, padding=1),<br>
    nn.ReLU(),<br>
    nn.Conv2d(output_channel, output_channel, kernel_size=3, stride=1, padding=1),<br>
    nn.ReLU(),<br>
    nn.MaxPool2d(kernel_size=2), <br>
    )<br>
<br>

## - FC block
<img src="./images/fc-block.png" alt="deeplearningwizard" style="width: 200px;"/>

- model = nn.Sequential(<br>
    nn.Linear(4096,4096),<br>
    nn.ReLU(),<br>
    nn.Lineard(4096,4096),<br>
    nn.ReLU(),<br>
    nn.Linear(4096,1000), <br>
    )<br>
    <br>

# Colab에서 GPU 활용하는 방법 <br>
<img src="./images/colab-gpu.png" alt="deeplearningwizard" style="width: 900px;"/>

<img src="./images/colab-gpu2.png" alt="deeplearningwizard" style="width: 900px;"/>